In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [1]:
!pip install transformers
# !/opt/bin/nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())

2.0.1+cu118
11.8
8700
True


In [ ]:
from google.colab import drive
drive.mount('/mntDrive')
import os
import json
import numpy as np

import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def get_passage_embds(model, date, passages_dir_prefix, para_embs_dir_prefix):
    model_name = model.split('/')[-1]
    passage_dir = passages_dir_prefix.format(date)
    para_embs_dir = para_embs_dir_prefix.format(model_name, date)

    if not os.path.exists(para_embs_dir):
        os.makedirs(para_embs_dir)

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModel.from_pretrained(model)
    model = model.to(device)

    for doc in os.listdir(passage_dir):
        print(doc)
        doc_fp = os.path.join(passage_dir, doc)
        save_fp = os.path.join(para_embs_dir, doc)
        with open(doc_fp, "r") as f:
            passages = json.load(f)

        ptxt_list = []
        pctxt_list = []
        for pid in range(len(passages["passages_text"])):
            ptxt_list.append(passages["passages_text"][pid]["ptxt"])
            if "passages_composite_text" in passages:
                pctxt_list.append(passages["passages_composite_text"][pid]["pctxt"])

        input_texts_ptxt = ["passage: " + i for i in ptxt_list]
        input_texts_pctxt = ["passage: " + i for i in pctxt_list]



        # Tokenize the input texts
        # ptxt_embs = []
        # batch_dict = tokenizer(input_texts_ptxt, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        # outputs = model(**batch_dict)
        # ptxt_embs = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).tolist()

        # pctxt_embs = []
        # batch_dict = tokenizer(input_texts_pctxt, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        # outputs = model(**batch_dict)
        # pctxt_embs = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).tolist()

        ptxt_embs = []
        start = 0
        windows = 2
        for i in input_texts_ptxt:
            batch_dict = tokenizer([i], max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
            outputs = model(**batch_dict)
            ptxt_emb = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).tolist()
            ptxt_embs.append(ptxt_emb[0])

        # while(start < len(input_texts_ptxt)):
        #     batch_dict = tokenizer(input_texts_ptxt[start: start+windows], max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        #     outputs = model(**batch_dict)
        #     ptxt_emb = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).tolist()
        #     ptxt_embs += ptxt_emb
        #     # print(ptxt_emb[0])
        #     # print(len(ptxt_embs))
        #     start += windows

        pctxt_embs = []
        # start = 0
        # while(start < len(input_texts_pctxt)):
        #     batch_dict = tokenizer(input_texts_pctxt[start: start+windows], max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        #     outputs = model(**batch_dict)
        #     pctxt_emb = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).tolist()
        #     pctxt_embs += pctxt_emb
        #     start += windows

        with open(save_fp, "w") as f:
            json.dump({"ptxt_embs": ptxt_embs, "pctxt_embs": pctxt_embs}, f)

In [ ]:
def query_embs(groundtruth_fp, save_fp):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')
    model = AutoModel.from_pretrained('intfloat/e5-large-v2')
    model = model.to(device)

    with open(groundtruth_fp, "r") as f:
        groundtruth = json.load(f)

        query_list = []
        for doc in groundtruth:
            for question in doc["questions"]:
                query = "query: " + question["query"]
                # encode query
                batch_dict = tokenizer([query], max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
                outputs = model(**batch_dict)
                q_ebm = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).tolist()

                question["qemb"] = q_ebm[0]

    with open(save_fp, "w") as f:
        json.dump(groundtruth, f)


In [ ]:
from transformers import AutoTokenizer, AutoModel
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model1 = "intfloat/e5-large-v2"
models = [model1]
for model_name in models:
  model = AutoModel.from_pretrained(model_name)
  total_params = sum(p.numel() for p in model.parameters())
  print(model_name, ": ", total_params)

In [ ]:
data_p =  '/mntDrive/MyDrive/Colab Notebooks/'
# passages_dir = data_p + "data/groundtruth/passages_proc_table_0708"
# para_embs_dir = data_p + "data/groundtruth/e5/para_proc_table_embs_0708"

# passages_dir = data_p + "data/groundtruth/passages_0708"
# para_embs_dir = data_p + "data/groundtruth/e5/para_embs_0708"

# passages_dir_prefix = data_p + "data/groundtruth/passages_chunk_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_chunk_{1}"

passages_dir_prefix = data_p + "data/groundtruth/passages_table2text_tid_{0}"
para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_table2text_{1}"

groundtruth_fp = data_p +  "data/groundtruth/qa_groundtruth.json"
query_emb =  data_p + "data/groundtruth/e5/query_emb.json"

# query_embs_groundtruth = data_p + "data/groundtruth/rocketqa/query_embs_groundtruth"

model = "intfloat/e5-large-v2"
date = "0708"

get_passage_embds(model, date, passages_dir_prefix, para_embs_dir_prefix)
# query_embs(groundtruth_fp, query_emb)

f5878758878cb3d640b25843695964fb.json
1c5aaf3f3c28e0d325567ae8c3f7b230.json
d7b3282af1c3f4a52958cb156b6cca98.json
5ad129a7811eecc376281f624eb26150.json
da3406d416687337a768901dd9e57002.json
60e406f5b6f6ae7121f8684fb3e4b05e.json
87d7fddb42cdd7554e61be3c70ebb97b.json
30e46d073b61081f322200a8a8502b41.json
e75ae83697fa878824fb62a150fa6d92.json
67a0f5b8398b39155067f20915efe454.json
3b849500b1ec818ec9b2dbdf9f114a72.json
6f99a700db4e43c1bc6abf015c2b677a.json
028ee763c1f90a74df4736ce7b5ffba4.json
ab0835f3e9b1c97629e6f370675a4a50.json
8c4cc201460e6703dc58f5d07900f34b.json
6d966ad0524ed9856b3f5191d05783d3.json
a62c00f8a92864a6e198b67593066a9d.json
39e4d8451914be777f7a671673803001.json
599d4ef0e5c755af465c0a9941bffb95.json
2f66d674d008a240d4a28a3f9349c66b.json
495a444939d6245f19d7fb8fb3c1470c.json
655b1f6aa9c1e5daf366f0ab9014687b.json
ad4a7e874ca9fb1684c3dd056b8cf966.json
bf53025254bf57a1a13b4a3c7cc841c1.json
4d41ea7a63b2d9b5cc3cb24ca6c7e9ac.json
2d440dca7029c64e7642bbc45e4cc906.json
0ba33bc0610e

In [ ]:
# def average_pool(last_hidden_states: Tensor,
#                  attention_mask: Tensor) -> Tensor:
#     last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
#     return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# device = "cuda:0" if torch.cuda.is_available() else "cpu"


# # Each input text should start with "query: " or "passage: ".
# # For tasks other than retrieval, you can simply use the "query: " prefix.
# input_texts = ['query: how much protein should a female eat',
#                'query: summit define',
#                "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#                "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]

# tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')
# model = AutoModel.from_pretrained('intfloat/e5-large-v2')

# # Tokenize the input texts
# batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
# model = model.to(device)
# outputs = model(**batch_dict)
# embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# # (Optionally) normalize embeddings
# embeddings = F.normalize(embeddings, p=2, dim=1)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist())
